In [1]:
import json
import torch
from transformers import AutoTokenizer, BertModel, LEDForConditionalGeneration
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from datasets import load_metric
import pandas as pd
from tqdm import tqdm

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


## Dataset

In [3]:
# load our augmented dataset json file
augmented_dataset = None
with open('augmented_test.json', 'r') as f:
    augmented_dataset = json.load(f)

# load our indices to use
sample_indices = pd.read_csv('cohere_sample_indices.csv').values.flatten()
dataset_small = [augmented_dataset[i] for i in sample_indices]

## Embeddings

In [4]:
# load our model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
def embed_documents(documents):
    doc_embeddings = []
    for doc in documents:
        inputs = tokenizer.encode_plus(doc, max_length=512, pad_to_max_length=True, return_tensors="pt")
        outputs = model(**inputs)

        last_hidden_states = outputs.last_hidden_state
        # we're going to average across the tokens to get the sentence embedding
        doc_embedding = torch.mean(last_hidden_states, dim=1)
        doc_embeddings.append(doc_embedding)
    
    return doc_embeddings

## Clustering

In [6]:
def cluster_documents(documents):
    doc_embeddings = embed_documents(documents)
    # turn our list of embeddings into a numpy matrix
    doc_embeddings = torch.cat(doc_embeddings).detach().numpy()
    
    # run PCA on the embeddings first
    pca = PCA(n_components=0.99)
    pca.fit(doc_embeddings)
    doc_embeddings = pca.fit_transform(doc_embeddings)

    # we are going to fine tune the number of clusters to use, based on silhoutte score
    best_score = -2
    best_kmeans = None

    for k in range(2, min(10, len(doc_embeddings))):
        kmeans = KMeans(n_clusters=k, random_state=0, n_init=10).fit(doc_embeddings)
        ss = silhouette_score(doc_embeddings, kmeans.labels_)
        if ss > best_score:
            best_score = ss
            best_kmeans = kmeans.labels_
        
    return best_kmeans

## Summarization

In [7]:
# load our model
TOKENIZER = AutoTokenizer.from_pretrained('allenai/PRIMERA')
MODEL = LEDForConditionalGeneration.from_pretrained('allenai/PRIMERA').to(device)

PAD_TOKEN_ID = TOKENIZER.pad_token_id
DOCSEP_TOKEN_ID = TOKENIZER.convert_tokens_to_ids("<doc-sep>")

In [8]:
def process_document(documents):
    input_ids_all=[]
    #### concat with global attention on doc-sep
    input_ids = []
    for doc in documents:
        input_ids.extend(
            TOKENIZER.encode(
                doc,
                truncation=True,
                max_length=min(4096 // len(documents), 4095),
            )[1:-1]
        )
        input_ids.append(DOCSEP_TOKEN_ID)
    input_ids = (
        [TOKENIZER.bos_token_id]
        + input_ids
        + [TOKENIZER.eos_token_id]
    )
    input_ids_all.append(torch.tensor(input_ids).to(device))
    input_ids_final = torch.nn.utils.rnn.pad_sequence(
        input_ids_all, batch_first=True, padding_value=PAD_TOKEN_ID
    ).to(device)

    return input_ids_final

"""
For each cluster, use each document in input. Output 512/k tokens for each cluster. Append them. 
"""
def process_clusters(data, clusters):
    documents = data['documents']
    n_clust = max(clusters)+1
    # create a list of lists, where each list is a cluster
    cluster_docs = [[] for _ in range(n_clust)]
    for i, doc in enumerate(documents):
        cluster_docs[clusters[i]].append(doc)

    # now run the summarization on each cluster
    cluster_summaries = []

    for cluster in cluster_docs:
        input_ids = process_document(cluster)
        
        # get the input ids and attention masks together
        global_attention_mask = torch.zeros_like(input_ids).to(input_ids.device)
        
        # put global attention on <s> token
        global_attention_mask[:, 0] = 1
        global_attention_mask[input_ids == DOCSEP_TOKEN_ID] = 1

        generated_ids = MODEL.generate(
            input_ids=input_ids,
            global_attention_mask=global_attention_mask,
            use_cache=True,
            max_length=512//n_clust,
            num_beams=5,
        )

        generated_str = TOKENIZER.batch_decode(
                generated_ids.tolist(), skip_special_tokens=True
            )
        cluster_summaries.append(generated_str)
    
    result={}
    # concatenate the summaries
    result['generated_summaries'] = " ".join([summary[0] for summary in cluster_summaries])
    result['gt_summaries']= data['summary']
    return result

In [9]:
# run our pipeline
results = []

for i in tqdm(range(0, len(dataset_small))):
    data = dataset_small[i]
    k_clusters = cluster_documents(data['documents'])
    results.append(process_clusters(data, k_clusters))

    # let's save the result every 50 points
    if i % 50 == 0:
        with open(f"results/final_pipeline_{i}.json", "w") as f:
            json.dump(results, f)
        

with open(f"results/final_pipeline_complete.json", "w") as f:
    json.dump(results, f)

  0%|                                                                                          | 0/200 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/envs/twitbits/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g.

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|▍                                                                                 | 1/200 [00:14<46:38, 14.06s/it]/opt/conda/envs/twitbits/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
  1%|▊                                                                                 | 2/200 [00:30<51:36, 15.64s/it]/opt/conda/envs/twitbits/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` 

  7%|█████▋                                                                           | 14/200 [03:29<49:35, 16.00s/it]/opt/conda/envs/twitbits/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
  8%|██████                                                                           | 15/200 [03:38<43:38, 14.15s/it]/opt/conda/envs/twitbits/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` 

 14%|██████████▉                                                                      | 27/200 [06:31<42:59, 14.91s/it]/opt/conda/envs/twitbits/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
 14%|███████████▎                                                                     | 28/200 [06:45<41:50, 14.59s/it]/opt/conda/envs/twitbits/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` 

...

 72%|█████████████████████████████████████████████████████████▏                      | 143/200 [32:49<12:48, 13.48s/it]/opt/conda/envs/twitbits/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
 72%|█████████████████████████████████████████████████████████▌                      | 144/200 [33:02<12:29, 13.38s/it]/opt/conda/envs/twitbits/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` 

 78%|██████████████████████████████████████████████████████████████▍                 | 156/200 [35:49<09:04, 12.38s/it]/opt/conda/envs/twitbits/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
 78%|██████████████████████████████████████████████████████████████▊                 | 157/200 [36:00<08:38, 12.06s/it]/opt/conda/envs/twitbits/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` 

 84%|███████████████████████████████████████████████████████████████████▌            | 169/200 [38:50<07:14, 14.01s/it]/opt/conda/envs/twitbits/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
 85%|████████████████████████████████████████████████████████████████████            | 170/200 [39:11<08:04, 16.13s/it]/opt/conda/envs/twitbits/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` 

 91%|████████████████████████████████████████████████████████████████████████▊       | 182/200 [41:51<03:39, 12.18s/it]/opt/conda/envs/twitbits/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
 92%|█████████████████████████████████████████████████████████████████████████▏      | 183/200 [42:02<03:21, 11.88s/it]/opt/conda/envs/twitbits/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` 

 98%|██████████████████████████████████████████████████████████████████████████████  | 195/200 [44:39<01:04, 12.82s/it]/opt/conda/envs/twitbits/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
 98%|██████████████████████████████████████████████████████████████████████████████▍ | 196/200 [44:50<00:49, 12.26s/it]/opt/conda/envs/twitbits/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` 